In [220]:
from pymongo import MongoClient
import pprint
import urllib.request

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import json
import time
import copy 

In [346]:
#pull census variables 
#storing census variable names(unreadable) with readable column names 
var_req = requests.get('https://api.census.gov/data/2018/acs/acs1/spp/variables.json').json()


col_name = {}
variables = var_req['variables']

for k, v in variables.items():
    col_name[k] = v['label']
    
#still need to clean up the column names and remove !! and other unnecssary data. 

In [347]:
#remove all the columns that have PR in them
clean_col_name = col_name.copy()
stat = 'PR'


for k, v in col_name.items():
    if stat in k:
        del clean_col_name[k]

In [381]:
#flexible pull request for api call, generates names for api url
def census_name_pull(name_dict, word_search):
    name_list = ['NAME','GEO_ID']


    for k, v in name_dict.items():
        if word_search in v:
            name_list.append(k)
    return sorted(name_list)

In [382]:
#pull columns for sex & age by county in state
sex_age_pull = census_name_pull(clean_col_name,'SEX AND AGE')


In [383]:
#pull columns for employment status by county in state
emp_stat_pull = census_name_pull(clean_col_name,'EMPLOYMENT STATUS')


In [384]:
#pull columns for industry by county in state
industry_pull = census_name_pull(clean_col_name,'INDUSTRY')


In [385]:
#pull columns for income by county in state
income_pull = census_name_pull(clean_col_name,'INCOME')


In [388]:
#connect to api_url for census per 
get_vars = [sex_age_pull, emp_stat_pull, industry_pull, income_pull]

def api_pull_request(get_vars):

    api_key = 'ce30285f0f2fedc096b5bf054c7d573ea6558309'
    base_url = 'https://api.census.gov/data/2018/acs/acs1/spp?'

    predicates = {}
    predicates['get'] = ','.join(get_vars)
    predicates['for'] = 'county:*'
    predicates['in'] = 'state:*'
    predicates['key'] = api_key
    
    r = requests.get(base_url, params=predicates)
    
    return r.json()

# api_url = '''https://api.census.gov/data/2018/acs/acs1/spp?get=NAME,GEO_ID,S0201_001E,S0201_006E,S0201_213E,S0201_194E&for=county:*&in=state:*&key=ce30285f0f2fedc096b5bf054c7d573ea6558309'''

In [394]:
#created sex and age pull request and dataframe
sex_age_data = api_pull_request(sex_age_pull)
sex_age_df = pd.DataFrame(sex_age_data)


In [395]:
#created employee status pull request and dataframe
emp_stat_data = api_pull_request(emp_stat_pull)
emp_stat_df = pd.DataFrame(emp_stat_data)


In [365]:
#created income pull request and dataframe
emp_stat_data = api_pull_request(emp_stat_pull)
emp_stat_df = pd.DataFrame(emp_stat_data)

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,S0201_006E,S0201_007E,S0201_008E,S0201_009E,S0201_010E,S0201_011E,S0201_012E,S0201_013E,S0201_014E,S0201_015E,...,S0201_030E,S0201_031E,S0201_032E,S0201_033E,S0201_034E,S0201_035E,S0201_036E,S0201_037E,state,county
1,597695,49.6,50.4,6,16.9,8.2,15.8,14.8,13.3,11.5,...,51.1,48.9,236957,49.6,50.4,80377,43.9,56.1,41,067
2,811880,49.6,50.4,5.5,13.3,7.8,18.8,16.5,13.1,11.5,...,49.8,50.2,333850,50.4,49.6,108862,44.1,55.9,41,051
3,1259428,49.5,50.5,6.5,15.5,8.5,17,13.7,12.2,12.6,...,50.8,49.2,484959,49.9,50.1,176860,44,56,27,053
4,550210,48.8,51.2,6.9,16.4,9.8,16.7,12.5,11.1,12.2,...,49.4,50.6,196719,49.2,50.8,79420,42.9,57.1,27,123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,648360,48.8,51.2,7.1,18.2,8.8,14.7,12.9,11.9,12.1,...,49.9,50.1,238874,48.7,51.3,93319,43,57,40,143
138,792582,49,51,7.5,18.2,9,15.6,12.9,11.3,11.9,...,50,50,285870,48.7,51.3,108277,43.5,56.5,40,109
139,1259201,49,51,5.5,15.5,8.1,13.3,12.5,14,14.3,...,51.5,48.5,513907,48.7,51.3,211586,43.8,56.2,26,125
140,1753893,48.2,51.8,6.5,17.1,8.7,14.2,11.7,13,13.3,...,49.4,50.6,667898,48,52,270261,42.3,57.7,26,163


In [153]:
#created industry pull request and dataframe
emp_stat_data = api_pull_request(emp_stat_pull)
emp_stat_df = pd.DataFrame(emp_stat_data)

200

In [154]:
data = r.json()

In [297]:
data2 = pd.DataFrame(data)
